In [1]:
import torch
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, AdamW
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}

In [3]:
# Set hyperparameters
weight_decay = 0.01        # Specified weight decay
learning_rate = 3e-5       # Specified learning rate
epochs = 3                 # Specified number of epochs
batch_size = 32            # Specified batch size

alltime = time.time()

# Define the Transformer-based model
class IoTClassifier(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(IoTClassifier, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 768)  # Map input to 768-dim
        self.transformer = BertModel.from_pretrained('bert-base-uncased')
        self.fc2 = torch.nn.Linear(768, num_classes)  # Final output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  
        x = self.transformer(inputs_embeds=x.unsqueeze(1)).last_hidden_state
        x = torch.mean(x, dim=1)  # Pooling
        x = self.fc2(x)  
        return x

# Define custom Dataset class
class IoTDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Feature selection and data loading
feature = ['ACK Flag Cnt', 'Active Max', 'Active Mean', 'Active Std', 'Bwd Header Len', 'Bwd IAT Min', 
           'Bwd IAT Tot', 'Bwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Min', 'FIN Flag Cnt', 
           'Flow Byts/s', 'Flow Duration', 'Flow IAT Max', 'Flow IAT Mean', 'Flow IAT Std', 'Fwd IAT Mean', 
           'Fwd IAT Min', 'Fwd IAT Std', 'Fwd IAT Tot', 'Fwd Pkt Len Max', 'Fwd Pkt Len Std', 'Fwd Pkts/s', 
           'Idle Max', 'Idle Mean', 'Idle Std', 'Init Bwd Win Byts', 'Pkt Len Max', 'Pkt Len Min', 
           'Pkt Len Var', 'Pkt Size Avg', 'Protocol', 'Src Port', 'Subflow Bwd Byts', 'Tot Fwd Pkts', 
           'TotLen Bwd Pkts', 'Label']

for f in file_list:
    print(f"Train: {file_list[f][0]} Test: {file_list[f][1]}")
    train = pd.read_csv(file_list[f][0], usecols=feature)
    test = pd.read_csv(file_list[f][1], usecols=feature)
    
    # Handle missing or invalid values
    train = train.replace(-9999, 0).fillna(0)
    test = test.replace(-9999, 0).fillna(0)
    
    # Separate features and labels
    X_train = train.drop(['Label'], axis=1)
    y_train = train['Label']
    
    # Encode labels and standardize features
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    # Separate features and labels for the test data
    X_test = test.drop(['Label'], axis=1)
    y_test = test['Label']
    
    # Encode labels and standardize features for the test data
    y_test = label_encoder.transform(y_test)
    X_test = scaler.transform(X_test)
    
    # Create DataLoaders using the specified batch_size
    train_dataset = IoTDataset(X_train, y_train)
    test_dataset = IoTDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Assign batch_size
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Initialize model, loss function, and optimizer with specified hyperparameters
    input_dim = X_train.shape[1]
    num_classes = len(label_encoder.classes_)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = IoTClassifier(input_dim, num_classes).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  # Assign learning_rate and weight_decay
    
    # Training loop with specified epochs
    num_epochs = epochs
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')
        
    train_time = float((time.time() - start_time))
    print(f"Training Time: {train_time:.2f} seconds")

    # Evaluation loop with F1-score calculation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_time = float((time.time() - start_time))
    print(f"Testing Time: {test_time:.2f} seconds")
    
    # Calculate F1-score
    f1 = f1_score(all_labels, all_preds, average='macro')
    print(f'Macro F1-Score: {f1:.4f}')
    
    # Optional: Calculate and print test accuracy
    correct = sum(p == l for p, l in zip(all_preds, all_labels))
    accuracy = 100 * correct / len(all_labels)
    print(f'Test Accuracy: {accuracy:.2f}%')

print(f"Total Time Elapsed: {time.time() - alltime:.2f} seconds")


Train: ./small/AD-S1.csv Test: ./small/DI-S1.csv
Epoch [1/3], Loss: 0.7825
Epoch [2/3], Loss: 0.4576
Epoch [3/3], Loss: 0.3910
Training Time: 8368.60 seconds
Testing Time: 8518.15 seconds
Macro F1-Score: 0.6019
Test Accuracy: 63.84%
Train: ./small/DI-S1.csv Test: ./small/AD-S1.csv
Epoch [1/3], Loss: 0.7968
Epoch [2/3], Loss: 0.4797
Epoch [3/3], Loss: 0.4168
Training Time: 4828.51 seconds
Testing Time: 4991.22 seconds
Macro F1-Score: 0.5554
Test Accuracy: 57.96%
Train: ./small/AD-S1.csv Test: ./small/DI-S2.csv
Epoch [1/3], Loss: 0.7897
Epoch [2/3], Loss: 0.4602
Epoch [3/3], Loss: 0.3970
Training Time: 5032.40 seconds
Testing Time: 5217.84 seconds
Macro F1-Score: 0.5255
Test Accuracy: 54.81%
Train: ./small/DI-S2.csv Test: ./small/AD-S1.csv
Epoch [1/3], Loss: 0.8429
Epoch [2/3], Loss: 0.5396
Epoch [3/3], Loss: 0.4586
Training Time: 5851.68 seconds
Testing Time: 6013.62 seconds
Macro F1-Score: 0.5603
Test Accuracy: 60.07%
Train: ./small/AD-S2.csv Test: ./small/DI-S2.csv
Epoch [1/3], Loss: 